In [1]:
# Import agentpy
import agentpy as ap

# Import basic libraries
import numpy as np
import matplotlib.pyplot as plt

# Visualization
import seaborn as sns
import IPython

Create environment

In [2]:
class Forest(ap.Model):

    # Configuration method
    def setup(self):
        '''
        This method works as a kind of constructor, executes when the instances are created
        Inputs:
         * dim: Forest dimension (dim x dim)
         * density: Forest tree density (0-1)
         * fire: Fired tree number
        Params are stored in self.p atributte as a dictionary (self.p['param'])
        '''

        # Create Trees
        amount = int(self.p['dim']*self.p['dim']*self.p['den'])
        self.trees = ap.AgentList(self, amount)

        # Create dimension environment (dim x dim)
        self.forest = ap.Grid(self, [self.p['dim']] * 2, track_empty=True)

        # Add agents to the environment
        self.forest.add_agents(self.trees, random=True, empty=True)

        # assign initial state to trees
        # (0: healthy, 4-9: In risk, 10: burned)
        self.trees.condition = 0

        # Initialize fired trees
        fired_trees = self.forest.agents[np.random.randint(self.p['dim'], size=self.p['fire']),
                                        np.random.randint(self.p['dim'], size=self.p['fire'])]
        
        fired_trees.condition = 4


    # Step method
    def step(self):
        '''
        This method is executed at each simulation step
        '''
        
        # Select trees in risk
        burning_trees = self.trees.select(self.trees.condition > 3)

        # Interaction trees with sorroundings
        for tree in burning_trees:
            
            for neighbor in self.forest.neighbors(tree):

                # Avoid burned trees
                if neighbor.condition < 10:
                    # Sorrounding trees get in risk
                    neighbor.condition += 1

                # Continue burning
                if tree.condition < 10:
                    tree.condition += 1

    # End method
    def end(self):
        '''
        This method is executed at the end of the simulation
        '''

In [3]:
# Aux function: Color RGB

def rgb2hex(colors):

    '''
    Convert a rgb tuple into a hex color format dictionary
    '''

    # Initialize dictionary
    labels = {} 


    # Convert RGB to Hex
    for i in range(len(colors)):
        hex = '#%02x%02x%02x' % (colors[i][0], colors[i][1], colors[i][2])

        # Add input to dictionary
        labels[i] = hex

    return labels

In [4]:
# Define status colors
colors_rgb = [[0,255,0], # Healthy
              [50,200,0], [100,150,0], [200,100,0], # In risk
              [255,50,0], [255,0,0], [200,0,0], [150,0,0], [100,0,0], [50,0,0], # Burning
              [0,0,0]] # Burned

# Convert RGB to Hex
colors_labels = rgb2hex(colors_rgb)
print(colors_labels)

{0: '#00ff00', 1: '#32c800', 2: '#649600', 3: '#c86400', 4: '#ff3200', 5: '#ff0000', 6: '#c80000', 7: '#960000', 8: '#640000', 9: '#320000', 10: '#000000'}


In [5]:
# Graph environment
def animation_plot(model, ax):


    # Get environment status
    attr_grid = model.forest.attr_grid('condition')

    # Graph status
    ap.gridplot(attr_grid, ax=ax, color_dict=colors_labels, convert=True)

In [6]:
# Define parameters
params = {
    'dim': 50,
    'den': 0.6,
    'fire': 2
}

# Create environment instance
model = Forest(params)

# Execute simulation
fig, ax = plt.subplots()
animation = ap.animate(model, fig, ax, animation_plot, steps=100)
IPython.display.HTML(animation.to_jshtml(fps=15))